# (1 / 2) Clustering

### i) One iteration of EM clustering

In [1]:
import numpy as np

#
# Preparation
#
observations = {
    'x1': np.array([1, 1]),
    'x2': np.array([-1, -1]),
    'x3': np.array([0.5, 0.55])
}

# Cluster parameters ("mu" = mean, "sigma" = covariance matrix)
mu1 = np.array([1, 1]) 
sigma1 = np.array([[1, 0], [0, 1]])
mu2 = np.array([-1, -1])
sigma2 = np.array([[1, 0], [0, 1]])

P_C1 = 0.6
P_C2 = 0.4

def multivariate_normal_likelihood(x, mu, sigma):
    d = len(x)
    coeff = 1 / ((2 * np.pi) ** (d / 2) * np.linalg.det(sigma) ** 0.5)
    exponent = -0.5 * (x - mu).T @ np.linalg.inv(sigma) @ (x - mu)
    return coeff * np.exp(exponent)

#
# Expectation (calculating the responsibilities)
#
results = {}
for name, x in observations.items():
    likelihood_C1 = multivariate_normal_likelihood(x, mu1, sigma1)
    likelihood_C2 = multivariate_normal_likelihood(x, mu2, sigma2)
    
    joint_C1 = P_C1 * likelihood_C1
    joint_C2 = P_C2 * likelihood_C2
    
    results[name] = {
        'likelihood_C1': likelihood_C1,
        'likelihood_C2': likelihood_C2,
        'joint_C1': joint_C1,
        'joint_C2': joint_C2
    }

for name, res in results.items():
    print(f"Results for {name}:")
    print(f"  Likelihood P(x | C=1): {res['likelihood_C1']:.6f}")
    print(f"  Likelihood P(x | C=2): {res['likelihood_C2']:.6f}")
    print(f"  Joint P(C=1, x): {res['joint_C1']:.6f}")
    print(f"  Joint P(C=2, x): {res['joint_C2']:.6f}")
    print()


#
# Maximization (updating the parameters considering the responsibilities)
#

# Find the maximum likelihoods and joint probabilities across all observations
max_likelihood_C1 = max(res['likelihood_C1'] for res in results.values())
max_likelihood_C2 = max(res['likelihood_C2'] for res in results.values())
max_joint_C1 = max(res['joint_C1'] for res in results.values())
max_joint_C2 = max(res['joint_C2'] for res in results.values())

print(f"Maximum Likelihood P(x | C=1): {max_likelihood_C1:.6f}")
print(f"Maximum Likelihood P(x | C=2): {max_likelihood_C2:.6f}")
print(f"Maximum Joint P(C=1, x): {max_joint_C1:.6f}")
print(f"Maximum Joint P(C=2, x): {max_joint_C2:.6f}")


Results for x1:
  Likelihood P(x | C=1): 0.159155
  Likelihood P(x | C=2): 0.002915
  Joint P(C=1, x): 0.095493
  Joint P(C=2, x): 0.001166

Results for x2:
  Likelihood P(x | C=1): 0.002915
  Likelihood P(x | C=2): 0.159155
  Joint P(C=1, x): 0.001749
  Joint P(C=2, x): 0.063662

Results for x3:
  Likelihood P(x | C=1): 0.126929
  Likelihood P(x | C=2): 0.015543
  Joint P(C=1, x): 0.076157
  Joint P(C=2, x): 0.006217

Maximum Likelihood P(x | C=1): 0.159155
Maximum Likelihood P(x | C=2): 0.159155
Maximum Joint P(C=1, x): 0.095493
Maximum Joint P(C=2, x): 0.063662


# (2 / 2) Software Experiment

### a) EM vs k-means

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics, datasets, tree, cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import davies_bouldin_score
from sklearn.mixture import GaussianMixture

NUM_CLUSTERS = 3
METRIC = 'euclidean'

# (0.) load the data
data = datasets.load_wine()
X,y= data.data, data.target

# (1.1) k-means 
kmeans_algo = cluster.KMeans(n_clusters=NUM_CLUSTERS,algorithm='lloyd',init='random',n_init=1)

# (1.2) Fit the model to the data
kmeans_model = kmeans_algo.fit(X)

# (1.3) Silhouette score
kmeans_model.cluster_centers_
labels = kmeans_model.labels_
silhouette_kmeans = metrics.silhouette_score(X, labels, metric=METRIC)
print(f"silhouette (k-means): {silhouette_kmeans}")

# (2.1) EM clusterin - Learn EM with multivariate Gaussian assumption (with 1 single run with starting point)
em_algo = GaussianMixture(n_components=NUM_CLUSTERS, covariance_type='full',n_init=1) 
em_model = em_algo.fit(X)

# (2.2) Silhouette score
labels_em= em_model.predict(X)
silhouette_emclustering = metrics.silhouette_score(X, labels_em, metric=METRIC)
print(f"silhouette (EM clustering): {silhouette_emclustering}")

# (3.) Comparing k-means with EM clustering (for wine dataset)
comparison_result = "k-means" if silhouette_kmeans > silhouette_emclustering else "EM clustering"
print(f"{comparison_result} is better!")

# (4.) Plot
plt.scatter(X[:, 0], X[:, 1], X[: ,2], c=labels_em)
plt.show()

### b) PCA with 2 components

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics, datasets, tree
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

NUM_COMPONENTS = 2

# (1.) load the data
data = datasets.load_wine()
X,y= data.data, data.target

# (2.) learn the transformation (components as linear combination of features)
pca = PCA(n_components=NUM_COMPONENTS)
X_pca = pca.fit(X).transform(X)
print("Components:\n",pca.components_)

# (3.) Let's plot it!
plt.scatter(X_pca[:,0], X_pca[:,1],c=y)
plt.show()
'''
The values overlap, so the three classes cannot be separated 
(the variance is not sufficient to perform classification in this dataset).
'''

# Extra (3D representation)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=y)
# plt.show()

# plt.scatter(X_pca[:,0], X_pca[:,1],c=y)
# plt.show()


### c) Clustering from PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets, cluster, metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score
from sklearn.mixture import GaussianMixture

NUM_COMPONENTS = 2
NUM_CLUSTERS = 3
METRIC = 'euclidean'

# (1.1) load the data
data = datasets.load_wine()
X, y = data.data, data.target

# (1.2) learn the transformation (components as linear combination of features)
pca = PCA(n_components=NUM_COMPONENTS)
X_pca = pca.fit(X).transform(X)
INPUT_DATASET = X_pca  # using previous dataset
# INPUT_DATASET = X

# (2.1) learning b) returned dataset (k-means)
kmeans_algo = cluster.KMeans(n_clusters=NUM_CLUSTERS, algorithm='lloyd', init='random', n_init=1)
kmeans_model = kmeans_algo.fit(INPUT_DATASET)  # using returned dataset from PCA

kmeans_model.cluster_centers_
labels = kmeans_model.labels_
silhouette_kmeans = metrics.silhouette_score(INPUT_DATASET, labels, metric=METRIC)
print(f"Silhouette (k-means): {silhouette_kmeans}")

plt.scatter(INPUT_DATASET[:, 0], INPUT_DATASET[:, 1], c=labels)
plt.show()

# (2.2) learning b) returned dataset (EM clustering)
em_algo = GaussianMixture(n_components=NUM_CLUSTERS, covariance_type='full', n_init=1)
em_model = em_algo.fit(INPUT_DATASET)

labels_em = em_model.predict(INPUT_DATASET)
silhouette_emclustering = metrics.silhouette_score(INPUT_DATASET, labels_em, metric=METRIC)
print(f"Silhouette (EM clustering): {silhouette_emclustering}")

# (3.) Scatter plot (EM clustering)
plt.scatter(INPUT_DATASET[:, 0], INPUT_DATASET[:, 1], c=labels_em)
plt.show()

# (3.) Comparing results (k-means VS EM clustering)
# 3. Comparing k-means with EM clustering (for wine dataset)
comparison_result = "k-means" if silhouette_kmeans > silhouette_emclustering else "EM clustering"
print(f"{comparison_result} is better!")

### d) PCA mapped data VS not mapped data

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics, datasets, cluster
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

# Flags to control the behaviours of the calculations
PCA_USED = True
NUM_CLUSTERS = 2
METRIC = 'euclidean'
NUM_COMPONENTS = 2

# Wrapper function (avoid repeated code)
def plot_data(input_data, c, title, label, PCA_USED):
    plt.scatter(input_data[:, 0], input_data[:, 1], c=c, cmap='plasma', marker='o', label=label)
    title += " on PCA-transformed Data" if PCA_USED else " without PCA"
    plt.title(title)
    if PCA_USED:
        plt.xlabel('Component 1')
        plt.ylabel('Component 2')
    plt.legend()
    plt.show()

# (1.1) load the data
data = datasets.load_breast_cancer()
X, y = data.data, data.target

if not PCA_USED:   # as untreated data
    INPUT_DATA = X
else:              # PCA Transformation
    pca = PCA(n_components=NUM_COMPONENTS)
    X_pca = pca.fit_transform(X)  # transformed data for consistent shape
    INPUT_DATA = X_pca

# (2.1) clustering (k-means) on PCA-transformed data
kmeans_algo = cluster.KMeans(n_clusters=NUM_CLUSTERS, algorithm='lloyd', init='random', n_init=1)
kmeans_model = kmeans_algo.fit(INPUT_DATA)
labels_kmeans = kmeans_model.labels_
silhouette_kmeans = metrics.silhouette_score(INPUT_DATA, labels_kmeans, metric=METRIC)
print(f"Silhouette (K-means): {silhouette_kmeans}")
plot_data(INPUT_DATA,labels_kmeans,'k-means clustering','K-means', PCA_USED)

# (2.2) EM Clustering on PCA-transformed data
em_algo = GaussianMixture(n_components=NUM_CLUSTERS, covariance_type='full', n_init=1)
em_model = em_algo.fit(INPUT_DATA)
labels_em = em_model.predict(INPUT_DATA)
silhouette_em = metrics.silhouette_score(INPUT_DATA, labels_em, metric=METRIC)
print(f"Silhouette (EM Clustering): {silhouette_em}")
plot_data(INPUT_DATA,labels_em,'EM clustering','EM', PCA_USED)

# (2.3) Comparing results (K-means vs EM Clustering)
comparison_result = "K-means" if silhouette_kmeans > silhouette_em else "EM Clustering"
print(f"{comparison_result} is better based on silhouette score!")

#TODO confirm that Davies' measure is NOT needed here